# Lista Pandas - Desempenho dos Alunos GDF

O dataset analisado aqui descreve o desempenho dos alunos da rede pública do Distrito Federal. Nele encontramos as notas bimestrais de todas as matérias oferecidas a cada aluno do GDF, juntamente com informações básicas sobre a escola, o curso e a série do aluno. A fonte dos dados é o portal de dados abertos do GDF.

Utilize o dataset para responder as questões abaixo. Coloque sua solução somente dentro do espaço delimitado para a resposta e não altere nenhum outro código disponibilizado. As bibliotecas para a resolução de todos os problemas já foram importadas na célula abaixo.

Obs.: Caso alguma delas não esteja instalada no seu ambiente execute o processo de instalção através de !pip install <nome_da_biblioteca>



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import requests as r
from zipfile import ZipFile
import io

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_data(rows=None,download=True) -> pd.core.frame.DataFrame:
    
    """ get_data(rows: int=None, download: bool=True) -> pandas.core.frame.DataFrame:
        
        - Retorna os dados para serem utilizados na resolução da lista. 
        - Utilize o parâmetro 'download' caso você já possua os dados na sua pasta de trabalho.
        
        
        ** Parâmetros **
        ________________
        
        rows : quantidade de linhas a serem lidas no dataset, padrão é None e representa todo o dataset.
        download : define se a função executará o processo de download e extração dos dados, padrão True.
        
        ** Exemplos **
        ________________
        
        >> df = get_data(rows=100)
        >> df.shape
        >> (100,23)
        
        
        >> df = get_data(rows=578)
        >> df.shape
        >> (578,23)
        
    """
    
    if download == True:
            
        file_zipped = r.get('http://dados.df.gov.br/dataset/b8436049-44e7-4224-95b4-224718a4b166/resource/3e654a9d-0647-4e39-930e-7cd07faec888/download/dados-abertos---desempenho-escolar20180515160111.zip')  
        z = ZipFile(io.BytesIO(file_zipped.content))
        z.extractall('.')
    
    return pd.read_csv('dados abertos - desempenho escolar_20180515_160111.csv',sep=';',encoding='latin-1',nrows=rows)

In [3]:
df = get_data()

# Perguntas

1. Quantos alunos diferentes estão matriculados no curso "Ensino Médio"?

In [4]:
new_df = df[df.curso=='Ensino Médio'].loc[:,['curso','cod_aluno']].drop_duplicates()
print(f'O curso Ensino Médio tem {new_df.shape[0]} alunos matriculados')
total_nota = df[(df.curso=='Ensino Médio') & (df.bimestre == 'resultado final')].loc[:,['curso','bimestre','cod_aluno']].drop_duplicates().shape[0]
print(f'Desses alunos, apenas {total_nota} tem nota Resultado Final')

O curso Ensino Médio tem 81712 alunos matriculados
Desses alunos, apenas 77122 tem nota Resultado Final


2. Levando em consideração somente os alunos do curso "Ensino Médio". Descreva a porcentagem de alunos que foram aprovados e reprovados (aluno aprovado é aquele que tirou 5 ou mais como nota final em todas as matérias cursadas) por série.

In [5]:
#Lista todas as materias de um dado aluno, para analisar o padrão de dados
#x = df[(x.disciplina=='Língua Estrangeira - Espanhol') & (df.bimestre=='resultado final')  & (df.curso=='Ensino Médio')  & (df.cod_aluno==-6302169521159888369)].reset_index(drop=True).loc[:,['disciplina','serie','cod_aluno','nota_bimestral','bimestre']]

def padroniza_nota(nota: float):
    if nota > 10.00:
        return round((nota / 10),2)
    else:
        return round(nota,2)

def valida_aprovacao(nota: float):
    if nota >= 5.00:
        return 'aprovado'
    else:
        return 'reprovado'

#retorna lista com as diferentes séries do Ensino Médio
disciplinas_not = ['Língua Estrangeira - Espanhol', 'Ensino Religioso','Projeto Interdisciplinar I','Projeto Interdisciplinar II','Projeto Interdisciplinar III']

for ser in df[df.curso=='Ensino Médio'].serie.unique():
    qtd_alunos_serie = df[(df.curso == 'Ensino Médio')&(df.serie == ser)&(df.bimestre=='resultado final')].cod_aluno.nunique()
    print('*************************************************')
    print(f'*  {ser}. Total de alunos {qtd_alunos_serie}')
    print('*************************************************')
    df_agg_series = (
        #filtra alunos da serie e somente nota final
        #OBS: lembrando que apenas 77122 de 81712 alunos tem essa nota
        df[(df.curso == 'Ensino Médio') & (df.serie == ser) & (df.bimestre=='resultado final')].loc[:,['nota_bimestral','disciplina','cod_aluno']]
        #ignora uma lista de disciplinas consideras não obrigatórias
        .query('disciplina != @disciplinas_not')            
        #troca notas nulas por 0
        .fillna(0)
        .replace(-1,0)
        #padroniza notas para ficarem todas no padrão de 0 a 10
        .assign(nota_padrao = lambda x: x.nota_bimestral.apply(padroniza_nota))
        #Deleta duplicados, considerando 'curso','bimestre','nota_bimestral','disciplina','cod_aluno'
        .drop_duplicates()  #shape 924975
        #define se naquela disciplina o aluno foi aprovado ou reprovado
        .assign(situacao = lambda x: x.nota_padrao.apply(valida_aprovacao)) #shape 924975
        .fillna(0)
        #agrupando por serie e cod_aluno e pegando o max(situacao) vai trazer na ordem: reprovado ou aprovado.
        #isso resolve a validação se ele foi aprovado em todas as matérias ou não
        .groupby(['cod_aluno'])
        .agg({'situacao': 'max'})
        #Como no groupby não usei as_index false então preciso do reset_index para retransformar a saída em um df
        #a saida normal do pandas pega a coluna agregadora e joga pro index.
        .fillna(0)
        .reset_index()
        .groupby(['situacao'],as_index = False)
        .agg(total_alunos=('cod_aluno','count'))
        .assign(Percentil = lambda x: x.total_alunos/qtd_alunos_serie) #shape 924975
    )
    print(df_agg_series)
    #print(df_agg_series.total_alunos.value_counts(normalize=True))


 #temos 81712 como resposta do exerc 1 mas temos só 77117 aqui. O motivo é que nem todos os alunos tem o bimestre
        # resultado final. Sendo assim, eles foram excluídos da totalização
        #Para resolver isso eria necessário considerar os bimestres 1, 2, 3 e 4 de cada aluno e fazer a media entre eles
        #validações
        #.query("cod_aluno == 9221709859008778696") #apro
        #.query("cod_aluno == -9222315045875238055") #apro
        #.query("situacao == 'aprovado'")
        #.query("cod_aluno == 9221396502265603763") #rep
        #.query("cod_aluno == 9223293934285108139") #rep


*************************************************
*  1ª Série. Total de alunos 32534
*************************************************
    situacao  total_alunos  Percentil
0   aprovado         20862   0.641237
1  reprovado         11667   0.358609
*************************************************
*  2ª Série. Total de alunos 24419
*************************************************
    situacao  total_alunos  Percentil
0   aprovado         16925   0.693108
1  reprovado          7494   0.306892
*************************************************
*  3ª Série. Total de alunos 20169
*************************************************
    situacao  total_alunos  Percentil
0   aprovado         17571   0.871188
1  reprovado          2598   0.128812


3. As coordenações regionais são instituições que cuidam das escolas públicas de uma determinada região. Busque dados populacionais de cada coordenação regional e demonstre se há alguma correlação entra a quantidade de alunos de cada coordenação e a população daquela cidade satélite.

In [6]:
# 1. Baixando os dados do html
df_populacao = pd.read_html('https://pt.m.wikipedia.org/wiki/Lista_de_regi%C3%B5es_administrativas_do_Distrito_Federal_por_popula%C3%A7%C3%A3o')
# pega apenas as 2 colunas que interessa
df_pop = df_populacao[0].loc[:,('Região administrativa','População')]
# Inclui CRE para ficar igual ao dado do dataframe
df_pop['Região administrativa'] = 'CRE - ' + df_pop['Região administrativa']
df_pop
#df_pop.loc[:,('Região administrativa','População')].to_dict()
#df_pop.to_dict()
#df_test = df_pop[:,'Região administrativa']


,Região administrativa,População
0,CRE - Ceilândia,349 955
1,CRE - Samambaia,231 942
2,CRE - Plano Piloto,217 073
3,CRE - Taguatinga,210 142
4,CRE - Planaltina,177 540
5,CRE - Guará,133 748
6,CRE - Gama,132 404
7,CRE - Recanto das Emas,131 058
8,CRE - Santa Maria,126 262
9,CRE - Águas Claras,117 346


In [7]:
#populacionais de cada coordenação regional e demonstre se há alguma correlação entra a quantidade de alunos de cada 
#coordenação e a população daquela cidade satélite.
def padroniza_nome(coord: str):
    return coord[6:].upper()

# Remove os espaços em branco e transforma str em int
def remove_whitespace(value):
  str_split = value.split()
  return int(''.join(str_split))

lista = (
    df[['coordenacao_regional','cod_aluno']] # shape 12779084
    .drop_duplicates() #shape 418286
    #.reset_index()
    .groupby('coordenacao_regional',as_index = False) #
    .agg(total_alunos=('cod_aluno','count'))  
    .merge(df_pop,left_on='coordenacao_regional',right_on='Região administrativa')    
    .assign(População = lambda x: x.População.apply(remove_whitespace))
    #.corr()

    #.loc[:,['total_alunos', 'População']]
    .corr()
)
#lista.info()
lista
#lista[['total_alunos','População']].corr()

,total_alunos,População
total_alunos,1.000000,0.887844
População,0.887844,1.000000


4. Levando em consideração somente os alunos do curso "Ensino Médio". Demonstre a porcentagem de alunos que obtiveram notas finais maiores ou iguais a 8 em Química, Física e Biologia, por coordenação Regional.

In [8]:
def valida_media_8(nota: float):
    if nota >= 8.00:
        return 1
    else:
        return 0

#considerar apenas essas disciplinas
disciplinas = ['Química', 'Física','Biologia']

df_base_8 = (
    #filtra dados
    df[(df.curso == 'Ensino Médio') & (df.bimestre=='resultado final')].loc[:,['cod_aluno','coordenacao_regional','disciplina','nota_bimestral']]
    #ignora uma lista de disciplinas consideras não obrigatórias
    .query('disciplina == @disciplinas')
    #troca notas nulas por 0
    .fillna(0)
    .replace(-1,0) #shape 231312
    #padroniza notas para ficarem todas no padrão de 0 a 10
    .assign(nota_padrao = lambda x: x.nota_bimestral.apply(padroniza_nota))
    #Deleta duplicados
    .drop_duplicates()  #não tem duplicados
    #agora filtra apenas notas maiores que 8
    #define se naquela disciplina o aluno tirou mais de 8
    .assign(situacao = lambda x: x.nota_padrao.apply(valida_media_8)) #shape 231312
    #agrupando por coordenação e aluno, para ver em quantas das 3 matérias o aluno tirou mais que 8 (situacao==1)
    .groupby(['coordenacao_regional','cod_aluno'])
    .agg({'situacao': 'sum'}) # 77116 alunos no total
    #.groupby(['coordenacao_regional','situacao'])
    #.agg({'cod_aluno': 'count'})
        #Como no groupby não usei as_index false então preciso do reset_index para retransformar a saída em um df
        #a saida normal do pandas pega a coluna agregadora e joga pro index.
       # .fillna(0)
    .reset_index()
    #.groupby(['coordenacao_regional'],as_index = False)
    #.agg(alunos_media_8=('cod_aluno','count'))
       # .assign(Percentil = lambda x: x.total_alunos/qtd_alunos_serie) #shape 924975
)

In [21]:
df_todo = df_base_8.groupby('coordenacao_regional').agg({'cod_aluno': 'count'})
df_media_8 = df_base_8[(df_base_8.situacao == 3)].groupby('coordenacao_regional').agg({'cod_aluno': 'count'})
df_media_8/df_todo

,cod_aluno
coordenacao_regional,
CRE - Brazlândia,0.112740
CRE - Ceilândia,0.149852
CRE - Gama,0.097118
CRE - Guará,0.168506
CRE - Núcleo Bandeirante,0.135548
CRE - Paranoá,0.176175
CRE - Planaltina,0.089280
CRE - Plano Piloto,0.094873
CRE - Recanto das Emas,0.134252


5. Qual coordenação regional tem a maior quantidade de alunos matriculados no turno diurno do curso "Ensino Médio".Considere somente as coordenações regionais que possuem um número acima da média de aprovados.

In [69]:
#calcula media de aprovação por coordenação
df_aprovacao_coord =  (
    df[(df.curso == 'Ensino Médio') & (df.turno=='Diurno') & (df.bimestre=='resultado final')].loc[:,['coordenacao_regional','nota_bimestral']]
    #troca notas nulas por 0
    .fillna(0)
    .replace(-1,0) #shape 231312
    #padroniza notas para ficarem todas no padrão de 0 a 10
    .assign(nota_padrao = lambda x: x.nota_bimestral.apply(padroniza_nota))
    #Deleta duplicados
    .drop_duplicates()  #não tem duplicados
    .groupby(['coordenacao_regional'],as_index = False)
    .agg(media=('nota_padrao','mean'))
    #.groupby('coordenacao_regional').agg(media={'nota_bimestral': 'mean'})
)
#df_aprovacao_coord

#calcula média geral de aprovação
media_aprovacao = df_aprovacao_coord.mean().sum()

#df_aprovacao_coord[df_aprovacao_coord.nota_bimestral]

In [70]:
df_aprovacao_coord[(df_aprovacao_coord.media > media_aprovacao)].nlargest(1, 'media')

,coordenacao_regional,media
10,CRE - Santa Maria,4.359333


6- Considerando somente alunos do curso "Ensino Médio". Os alunos do turno diurno perfomam melhor que os alunos do turno noturno nas matérias de química, física e biologia. (Verdadeiro ou Falso, demonstre com dados).

In [71]:
disciplinas = ['Química', 'Física','Biologia']

#calcula media de aprovação por turno
df_analise_turno =  (
    df[(df.curso == 'Ensino Médio') & (df.bimestre=='resultado final')].loc[:,['turno','disciplina','cod_aluno','nota_bimestral']]
    #ignora uma lista de disciplinas consideras não obrigatórias
    .query('disciplina == @disciplinas')
    #troca notas nulas por 0
    .fillna(0)
    .replace(-1,0) 
    #padroniza notas para ficarem todas no padrão de 0 a 10
    .assign(nota_padrao = lambda x: x.nota_bimestral.apply(padroniza_nota))
    #Deleta duplicados
    .drop_duplicates() 
    .groupby(['turno'])
    .agg(media=('nota_padrao','mean'))
)
df_analise_turno.reset_index()

,turno,media
0,Diurno,5.913894
1,Noturno,7.202392


RESPOSTA: Como podemos ver acima a média dos alunos no turno Noturno, considerando apenas as 3 disciplincas ('Química', 'Física','Biologia') é maior que a média dos alunos no turno Diurno, com as mesmas variáveis de análise.